# ITI Mass Mean Shift β Analysis — Final (α=15, K=48, COM, fixed thresholds)


**Plan summary (matched to `validate_2fold`):**
- **COM directions**, **K=48 heads**, **α=15** fixed, greedy decoding.
- **β ∈ {0,1,2,5,10}**; negatives = FALSE.
- **2‑fold CV**: for each fold, compute directions on train (with an internal val from train), evaluate on held‑out. **Pool both held‑out folds** for analysis.
- **Judges:** use **binary** `llama3_8B_instruct GPT-judge` (truth) and `llama3_8B_instruct GPT-info` (informative). If absent, fallback to **prob** (`judge_truth_prob`, `judge_info_prob`) with **fixed 0.5 threshold** (no calibration).
- **Seeds:** parse from filenames; β=1 may have multiple seeds → show replication table (descriptive). Main tests **paired over questions**, not seeds.
- **Metrics:** TI (primary), H, U; 4‑bucket; **pairwise vs β=1** (McNemar ΔTI/ΔH + Holm), Bowker (4‑bucket), trends (Cochran–Armitage). Optional MC1/MC2/CE/KL from `summary_dump`.


## 1) Configuration

In [15]:

# Data sources (per-item)
ASSESSED_DIR_CANDIDATES = [
    "validation/results_dump/assessed_dump/",
    "validation/results_dump/assesed_dump/",
]

# Aggregate summaries (optional)
SUMMARY_GLOBS = [
    "validation/results_dump/summary_dump/*_com_true_info.csv",
    "validation/results_dump/summary_dump/*_com.csv",
]

# Fixed experiment parameters
ALPHA_FIXED = 15
K_FIXED = 48
BASELINE_BETA = 1
BETA_EXPECTED = [0, 1, 2, 5, 10]

# Exact schema expected for assessed dump files
EXPECTED_COLUMNS = [
    'Type', 'Category', 'Question', 'Best Answer', 'Best Incorrect Answer',
    'Correct Answers', 'Incorrect Answers', 'Source',
    'llama3_8B_instruct', 'llama3_8B_instruct lprob max', 'llama3_8B_instruct lprob diff',
    'llama3_8B_instruct lprob scores-true', 'llama3_8B_instruct lprob scores-false',
    'llama3_8B_instruct MC1', 'llama3_8B_instruct MC2', 'llama3_8B_instruct MC3',
    'llama3_8B_instruct GPT-judge', 'judge_truth_logit', 'judge_truth_prob',
    'llama3_8B_instruct GPT-judge acc',
    'llama3_8B_instruct GPT-info', 'judge_info_logit', 'judge_info_prob',
    'llama3_8B_instruct GPT-info acc',
    'beta', 'folds', 'seed', "filename"
]
# Judge columns (binary first; prob fallback)
TRUTH_BIN_COL = "llama3_8B_instruct GPT-judge"
INFO_BIN_COL  = "llama3_8B_instruct GPT-info"
TRUTH_PROB_COL = "judge_truth_prob"
INFO_PROB_COL  = "judge_info_prob"
FIXED_TRUTH_THRESHOLD = 0.5
FIXED_INFO_THRESHOLD  = 0.5

# Folds
FOLDS_TO_POOL = [0, 1]  # pool both held-out folds

# Seeds
PRIMARY_SEED_FOR_TESTS = None  # None => auto-pick smallest observed for β=1

# Stats
RNG_SEED = 2025
N_BOOT = 10_000
ALPHA_LEVEL = 0.05
U_TOLERANCE_Y = 0.0

# Imports
import os, glob, re, math, random, warnings
import numpy as np, pandas as pd
from collections import Counter
from scipy.stats import binomtest, chi2, norm
import plotly.graph_objects as go

warnings.filterwarnings("ignore")
random.seed(RNG_SEED); np.random.seed(RNG_SEED)


## 2) Load per-item CSVs (pool both folds), assert schema, parse β/folds/seed

In [16]:
def extract_beta_from_filename(filename):
    m = re.search(r'(?:^|[_-])beta_([0-9]+(?:\.[0-9]+)?)', os.path.basename(filename))
    # if beta is not found, return 1.0 (default)
    if m is None:
        return 1.0
    return float(m.group(1)) if m else None

def extract_folds_from_filename(filename):
    m = re.search(r'(?:^|[_-])fold_(\d+)', os.path.basename(filename))
    return int(m.group(1)) if m else None

def extract_seed_from_filename(filename):
    m = re.search(r'(?:^|[_-])seed_(\d+)', os.path.basename(filename))
    return int(m.group(1)) if m else None

# Pick a directory that exists
ASSESSED_DIR = None
for cand in ASSESSED_DIR_CANDIDATES:
    if os.path.isdir(cand) and len(glob.glob(os.path.join(cand, "*.csv"))) > 0:
        ASSESSED_DIR = cand; break
assert ASSESSED_DIR is not None, "No assessed/answer dump directory found."

csv_files = sorted(glob.glob(os.path.join(ASSESSED_DIR, "*.csv")))
print(f"Found {len(csv_files)} CSV files in {ASSESSED_DIR}")

frames = []
expected_set = set(EXPECTED_COLUMNS)
for path in csv_files:
    df = pd.read_csv(path)
    # Assert exact schema before we add seed/beta/folds
    # Parse and attach metadata
    seed = extract_seed_from_filename(path)
    beta = extract_beta_from_filename(path) 
    fold = extract_folds_from_filename(path)
    df["filename"] = os.path.basename(path)  # keep for reference
    df["seed"] = seed
    df["beta"] = beta
    df["folds"] = fold
        
    cols_set = set(df.columns.tolist())
    assert cols_set == expected_set, f"Schema mismatch in {os.path.basename(path)}.\nExpected: {sorted(expected_set)}\nGot: {sorted(cols_set)}"
    
    frames.append(df)

raw_df = pd.concat(frames, ignore_index=True)
print("Loaded rows:", len(raw_df))

# Pool both folds (held-out from validate_2fold)
df = raw_df[raw_df["folds"].isin(FOLDS_TO_POOL)].copy()
print("Rows after pooling folds", FOLDS_TO_POOL, ":", len(df))

# Determine judge mode
use_binary = (TRUTH_BIN_COL in df.columns) and (INFO_BIN_COL in df.columns)
print("Using", "binary judge columns" if use_binary else "probability columns with fixed 0.5 thresholds")

# Beta order (intersect expected with present)
betas_present = sorted(df["beta"].dropna().unique().tolist(), key=lambda x: BETA_EXPECTED.index(x) if x in BETA_EXPECTED else 999)
BETA_ORDER = [b for b in BETA_EXPECTED if b in betas_present]
assert BASELINE_BETA in BETA_ORDER, f"Baseline β={BASELINE_BETA} not present."

# Seed handling: choose primary seed for β=1 deterministically for tests
seeds_b1 = sorted(df.loc[df["beta"]==BASELINE_BETA, "seed"].dropna().unique().tolist())
PRIMARY_SEED = (seeds_b1[0] if seeds_b1 else None) if PRIMARY_SEED_FOR_TESTS is None else PRIMARY_SEED_FOR_TESTS
print("β=1 seeds found:", seeds_b1, "→ primary seed used for tests:", PRIMARY_SEED)


Found 10 CSV files in validation/results_dump/assesed_dump/
Loaded rows: 3950
Rows after pooling folds [0, 1] : 3950
Using binary judge columns
β=1 seeds found: [42] → primary seed used for tests: 42


## 3) Build per‑β evaluation frames, collapse seeds deterministically, pair by `Question`

In [17]:

COL_QID = "Question"

def prepare_eval_beta(beta):
    sub = df[df["beta"] == float(beta)].copy()
    # If multiple seeds, pick smallest seed for determinism
    if "seed" in sub.columns and sub["seed"].notna().any():
        seeds = sorted(sub["seed"].dropna().unique().tolist())
        chosen = PRIMARY_SEED if (beta == BASELINE_BETA and PRIMARY_SEED is not None and PRIMARY_SEED in seeds) else seeds[0]
        sub = sub[sub["seed"] == chosen].copy()
    # Derive T_bin / I_bin
    if use_binary:
        sub["T_bin"] = sub[TRUTH_BIN_COL].astype(int)
        sub["I_bin"] = sub[INFO_BIN_COL].astype(int)
    else:
        sub["T_bin"] = (sub[TRUTH_PROB_COL].astype(float) >= FIXED_TRUTH_THRESHOLD).astype(int)
        sub["I_bin"] = (sub[INFO_PROB_COL].astype(float)  >= FIXED_INFO_THRESHOLD).astype(int)
    # 4-bucket
    def to_bucket_row(row):
        t, i = int(row["T_bin"]), int(row["I_bin"])
        if t==1 and i==1: return "T&I"
        if t==1 and i==0: return "T&¬I"
        if t==0 and i==1: return "¬T&I"
        return "¬T&¬I"
    sub["bucket"] = sub.apply(to_bucket_row, axis=1)
    return sub

eval_by_beta = {b: prepare_eval_beta(b) for b in BETA_ORDER}

# Pair items across β using intersection of questions
common_qids = None
for b, d in eval_by_beta.items():
    s = set(d[COL_QID].tolist())
    common_qids = s if common_qids is None else (common_qids & s)
paired_qids = sorted(list(common_qids)) if common_qids else []
print("Paired questions across all β:", len(paired_qids))
assert len(paired_qids) > 0, "No overlapping questions across β conditions."

# Build map for fast access
by_beta_q = {b: {row[COL_QID]: row for _, row in d.iterrows() if row[COL_QID] in paired_qids}
             for b, d in eval_by_beta.items()}


Paired questions across all β: 790


## 4) Metrics, paired CIs, pairwise tests vs β=1, trend tests

In [4]:

# Indicators
is_TI = lambda row: int(row["T_bin"]) == 1 and int(row["I_bin"]) == 1
is_H  = lambda row: int(row["T_bin"]) == 0 and int(row["I_bin"]) == 1
is_T  = lambda row: int(row["T_bin"]) == 1
is_I  = lambda row: int(row["I_bin"]) == 1

df_map = by_beta_q
qids = paired_qids

# Paired bootstrap
def paired_bootstrap_rate(df_map, indicator_fn, qids, n_boot=N_BOOT, seed=RNG_SEED):
    rng = np.random.default_rng(seed)
    rates = {}
    for b, rows in df_map.items():
        vals = np.array([indicator_fn(rows[q]) for q in qids])
        rates[b] = float(vals.mean())
    boots = {b: [] for b in df_map.keys()}
    qids_arr = np.array(qids, dtype=object)
    for _ in range(n_boot):
        res_q = rng.choice(qids_arr, size=len(qids_arr), replace=True)
        for b, rows in df_map.items():
            vals = np.array([indicator_fn(rows[q]) for q in res_q])
            boots[b].append(vals.mean())
    ci = {b: (float(np.percentile(boots[b], 2.5)), float(np.percentile(boots[b], 97.5))) for b in boots}
    return rates, ci

def paired_bootstrap_diff(df_map, indicator_fn, qids, base_beta=BASELINE_BETA, n_boot=N_BOOT, seed=RNG_SEED):
    rng = np.random.default_rng(seed)
    base_vals = np.array([indicator_fn(df_map[base_beta][q]) for q in qids])
    diffs = {}
    qids_arr = np.array(qids, dtype=object)
    boots = {b: [] for b in df_map.keys()}
    for b in df_map.keys():
        vals = np.array([indicator_fn(df_map[b][q]) for q in qids])
        diffs[b] = float(vals.mean() - base_vals.mean())
    for _ in range(n_boot):
        res_q = rng.choice(qids_arr, size=len(qids_arr), replace=True)
        base_vals_b = np.array([indicator_fn(df_map[base_beta][q]) for q in res_q])
        for b in df_map.keys():
            vals_b = np.array([indicator_fn(df_map[b][q]) for q in res_q])
            boots[b].append(vals_b.mean() - base_vals_b.mean())
    ci = {b: (float(np.percentile(boots[b], 2.5)), float(np.percentile(boots[b], 97.5))) for b in boots}
    return diffs, ci

def wilson_interval(k, n, alpha=ALPHA_LEVEL):
    if n == 0: return (0.0, 0.0)
    z = norm.ppf(1 - alpha/2); p = k / n
    denom = 1 + z**2/n
    center = (p + z**2/(2*n)) / denom
    half = (z * math.sqrt((p*(1-p) + z**2/(4*n))/n)) / denom
    return (max(0.0, center - half), min(1.0, center + half))

def mcnemar_exact(n01, n10):
    n = n01 + n10
    if n == 0: return 1.0
    k = min(n01, n10)
    return binomtest(k, n=n, p=0.5, alternative="two-sided").pvalue

def bowker_test(conf_mat):
    k = conf_mat.shape[0]; stat = 0.0; df_b = 0
    for i in range(k):
        for j in range(i+1, k):
            nij = conf_mat[i, j]; nji = conf_mat[j, i]
            denom = nij + nji
            if denom > 0:
                stat += (nij - nji)**2 / denom; df_b += 1
    pval = 1 - chi2.cdf(stat, df_b) if df_b > 0 else 1.0
    return stat, df_b, pval

def holm_bonf(pdict):
    items = sorted(pdict.items(), key=lambda kv: kv[1])
    m = len(items); adj = {}
    for i, (name, p) in enumerate(items, start=1):
        adj[name] = min(1.0, (m - i + 1) * p)
    for i in range(1, m):
        k1, k2 = items[i-1][0], items[i][0]
        adj[k2] = max(adj[k2], adj[k1])
    return adj

# Rates and CIs
rates_TI, ci_TI = paired_bootstrap_rate(df_map, is_TI, qids)
rates_H,  ci_H  = paired_bootstrap_rate(df_map, is_H,  qids)
rates_T,  ci_T  = paired_bootstrap_rate(df_map, is_T,  qids)
rates_I,  ci_I  = paired_bootstrap_rate(df_map, is_I,  qids)

# U vs β=1
base = df_map[BASELINE_BETA]
base_TI_q = [q for q in qids if is_TI(base[q])]
U, U_ci, U_counts = {}, {}, {}
for b in BETA_ORDER:
    if b == BASELINE_BETA:
        U[b], U_ci[b], U_counts[b] = 0.0, (0.0, 0.0), (0, len(base_TI_q)); continue
    k = sum(1 for q in base_TI_q if (int(df_map[b][q]["T_bin"])==1 and int(df_map[b][q]["I_bin"])==0))
    n = len(base_TI_q)
    U[b] = k / n if n>0 else 0.0
    U_ci[b] = wilson_interval(k, n, alpha=ALPHA_LEVEL)
    U_counts[b] = (k, n)

# Pairwise vs β=1
def pairwise_mcnemar(df_map, qids, indicator_fn, base_beta=BASELINE_BETA):
    results = {}
    base_vals = np.array([indicator_fn(df_map[base_beta][q]) for q in qids], dtype=int)
    for b in [bb for bb in BETA_ORDER if bb != base_beta]:
        vals = np.array([indicator_fn(df_map[b][q]) for q in qids], dtype=int)
        n01 = int(np.sum((base_vals==0) & (vals==1)))
        n10 = int(np.sum((base_vals==1) & (vals==0)))
        p = mcnemar_exact(n01, n10)
        results[b] = dict(n01=n01, n10=n10, p=p)
    return results

mcn_TI = pairwise_mcnemar(df_map, qids, is_TI)
mcn_H  = pairwise_mcnemar(df_map, qids, is_H)

rd_TI, rd_TI_ci = paired_bootstrap_diff(df_map, is_TI, qids, base_beta=BASELINE_BETA)
rd_H,  rd_H_ci  = paired_bootstrap_diff(df_map, is_H,  qids, base_beta=BASELINE_BETA)

# 4-bucket Bowker
bucket_levels = ["T&I","T&¬I","¬T&I","¬T&¬I"]
def confusion_buckets(base_df, tgt_df, qids):
    mat = np.zeros((4,4), dtype=int)
    idx = {lab:i for i, lab in enumerate(bucket_levels)}
    for q in qids:
        a = base_df[q]["bucket"]; b = tgt_df[q]["bucket"]
        mat[idx[a], idx[b]] += 1
    return mat

bowker = {}
for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
    cm = confusion_buckets(df_map[BASELINE_BETA], df_map[b], qids)
    stat, df_b, p_b = bowker_test(cm)
    bowker[b] = dict(stat=stat, df=df_b, p=p_b)

# Holm-Bonferroni adjusted p-values
adj_TI = holm_bonf({b: mcn_TI[b]["p"] for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]})
adj_H  = holm_bonf({b: mcn_H[b]["p"]  for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]})

# Trend tests (Cochran–Armitage approx)
def aggregate_counts(df_map, qids, indicator_fn):
    succ = []; tots = []
    for b in BETA_ORDER:
        vals = np.array([indicator_fn(df_map[b][q]) for q in qids], dtype=int)
        succ.append(vals.sum()); tots.append(len(vals))
    return succ, tots

def cochran_armitage(successes, totals, scores):
    successes = np.asarray(successes, dtype=float); totals = np.asarray(totals, dtype=float)
    scores = np.asarray(scores, dtype=float); N = totals.sum(); p = successes.sum()/N
    num = np.sum(scores * (successes - totals*p))
    denom = math.sqrt(p*(1-p) * ( np.sum(scores**2 * totals) - (np.sum(scores*totals)**2) / N ))
    if denom==0: return 0.0, 1.0
    z = num/denom; p_two = 2*(1 - norm.cdf(abs(z))); return z, p_two

scores = np.array(BETA_ORDER, dtype=float)
succ_TI, tot_TI = aggregate_counts(df_map, qids, is_TI)
succ_H,  tot_H  = aggregate_counts(df_map, qids, is_H)
z_ti, p_ti = cochran_armitage(succ_TI, tot_TI, scores)
z_h,  p_h  = cochran_armitage(succ_H,  tot_H,  scores)

trend_df = pd.DataFrame({"metric":["TI","H"], "z":[z_ti, z_h], "p_two_sided":[p_ti, p_h]}).set_index("metric")

# Summary & pairwise tables
rows = []
for b in BETA_ORDER:
    rows.append(dict(beta=b,
                     TI=rates_TI[b], TI_lo=ci_TI[b][0], TI_hi=ci_TI[b][1],
                     H=rates_H[b],  H_lo=ci_H[b][0],   H_hi=ci_H[b][1],
                     T=rates_T[b],  T_lo=ci_T[b][0],   T_hi=ci_T[b][1],
                     I=rates_I[b],  I_lo=ci_I[b][0],   I_hi=ci_I[b][1],
                     U=U[b],        U_lo=U_ci[b][0],   U_hi=U_ci[b][1],
                     U_k=U_counts[b][0], U_n=U_counts[b][1]))
summary_df = pd.DataFrame(rows).set_index("beta").sort_index()

pair_rows = []
for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
    pair_rows.append(dict(beta=b,
                          TI_RD=rd_TI[b], TI_CI_lo=rd_TI_ci[b][0], TI_CI_hi=rd_TI_ci[b][1],
                          TI_n01=mcn_TI[b]["n01"], TI_n10=mcn_TI[b]["n10"], TI_p=mcn_TI[b]["p"], TI_p_holm=adj_TI[b],
                          H_RD=rd_H[b],  H_CI_lo=rd_H_ci[b][0], H_CI_hi=rd_H_ci[b][1],
                          H_n01=mcn_H[b]["n01"],  H_n10=mcn_H[b]["n10"], H_p=mcn_H[b]["p"], H_p_holm=adj_H[b]))
pairwise_df = pd.DataFrame(pair_rows).set_index("beta").sort_index()

print("Tables ready: metrics by β, pairwise vs β=1, trend tests")
display(summary_df.round(4)); display(pairwise_df.round(4)); display(trend_df)


Tables ready: metrics by β, pairwise vs β=1, trend tests


,TI,TI_lo,TI_hi,H,H_lo,H_hi,T,T_lo,T_hi,I,I_lo,I_hi,U,U_lo,U_hi,U_k,U_n
beta,,,,,,,,,,,,,,,,,
0,0.5025,0.4684,0.5380,0.0810,0.0620,0.1000,0.9190,0.9000,0.9380,0.5835,0.5494,0.6177,0.0295,0.0169,0.0508,12,407
1,0.5152,0.4810,0.5494,0.0797,0.0608,0.0987,0.9203,0.9013,0.9392,0.5949,0.5608,0.6291,0.0000,0.0000,0.0000,0,407
2,0.5190,0.4848,0.5532,0.0797,0.0608,0.0987,0.9203,0.9013,0.9392,0.5987,0.5646,0.6329,0.0270,0.0152,0.0477,11,407
5,0.5215,0.4861,0.5570,0.0696,0.0532,0.0873,0.9304,0.9127,0.9468,0.5911,0.5570,0.6253,0.0909,0.0667,0.1228,37,407
10,0.5038,0.4696,0.5392,0.0532,0.0380,0.0696,0.9456,0.9291,0.9608,0.5570,0.5228,0.5924,0.1400,0.1097,0.1771,57,407


,TI_RD,TI_CI_lo,TI_CI_hi,TI_n01,TI_n10,TI_p,TI_p_holm,H_RD,H_CI_lo,H_CI_hi,H_n01,H_n10,H_p,H_p_holm
beta,,,,,,,,,,,,,,
0,-0.0127,-0.0278,0.0025,13,23,0.1325,0.53,0.0013,-0.0101,0.0127,12,11,1.0000,1.0000
2,0.0038,-0.0114,0.0203,22,19,0.7552,1.00,0.0000,-0.0114,0.0114,10,10,1.0000,1.0000
5,0.0063,-0.0203,0.0329,57,52,0.7018,1.00,-0.0101,-0.0266,0.0063,19,27,0.3020,0.9060
10,-0.0114,-0.0405,0.0177,63,72,0.4913,1.00,-0.0266,-0.0456,-0.0076,19,40,0.0086,0.0346


,z,p_two_sided
metric,,
TI,-0.171013,0.864214
H,-2.532615,0.011322


In [5]:
pretty_summary = (summary_df
    .rename_axis("β")
    .rename(columns={
        "TI":"True×Informative (TI)",
        "H":"Hallucinations (H)",
        "T":"Truth rate (T)",
        "I":"Informativeness (I)",
        "U":"Unnecessary abstention (U)",
        "TI_lo":"TI CI lo","TI_hi":"TI CI hi",
        "H_lo":"H CI lo","H_hi":"H CI hi",
        "T_lo":"T CI lo","T_hi":"T CI hi",
        "I_lo":"I CI lo","I_hi":"I CI hi",
        "U_lo":"U CI lo","U_hi":"U CI hi",
        "U_k":"U numerator","U_n":"U denominator"
    }))
display(pretty_summary)

pretty_pairs = (pairwise_df
    .rename_axis("β vs baseline (β=1)")
    .rename(columns={
        "TI_RD":"ΔTI (risk diff)",
        "TI_CI_lo":"ΔTI CI lo","TI_CI_hi":"ΔTI CI hi",
        "TI_n01":"TI n01 (0→1)","TI_n10":"TI n10 (1→0)",
        "TI_p":"TI McNemar p","TI_p_holm":"TI Holm-adj p",
        "H_RD":"ΔH (risk diff)",
        "H_CI_lo":"ΔH CI lo","H_CI_hi":"ΔH CI hi",
        "H_n01":"H n01 (0→1)","H_n10":"H n10 (1→0)",
        "H_p":"H McNemar p","H_p_holm":"H Holm-adj p"
    }))
display(pretty_pairs)



,True×Informative (TI),TI CI lo,TI CI hi,Hallucinations (H),H CI lo,H CI hi,Truth rate (T),T CI lo,T CI hi,Informativeness (I),I CI lo,I CI hi,Unnecessary abstention (U),U CI lo,U CI hi,U numerator,U denominator
β,,,,,,,,,,,,,,,,,
0,0.502532,0.468354,0.537975,0.081013,0.062025,0.100000,0.918987,0.900000,0.937975,0.583544,0.549367,0.617722,0.029484,0.016945,0.050822,12,407
1,0.515190,0.481013,0.549367,0.079747,0.060759,0.098734,0.920253,0.901266,0.939241,0.594937,0.560759,0.629114,0.000000,0.000000,0.000000,0,407
2,0.518987,0.484810,0.553165,0.079747,0.060759,0.098734,0.920253,0.901266,0.939241,0.598734,0.564557,0.632911,0.027027,0.015157,0.047742,11,407
5,0.521519,0.486076,0.556962,0.069620,0.053165,0.087342,0.930380,0.912658,0.946835,0.591139,0.556962,0.625316,0.090909,0.066674,0.122794,37,407
10,0.503797,0.469620,0.539241,0.053165,0.037975,0.069620,0.945570,0.929114,0.960759,0.556962,0.522785,0.592405,0.140049,0.109689,0.177140,57,407


,ΔTI (risk diff),ΔTI CI lo,ΔTI CI hi,TI n01 (0→1),TI n10 (1→0),TI McNemar p,TI Holm-adj p,ΔH (risk diff),ΔH CI lo,ΔH CI hi,H n01 (0→1),H n10 (1→0),H McNemar p,H Holm-adj p
β vs baseline (β=1),,,,,,,,,,,,,,
0,-0.012658,-0.027848,0.002532,13,23,0.132498,0.529993,0.001266,-0.010127,0.012658,12,11,1.000000,1.000000
2,0.003797,-0.011392,0.020253,22,19,0.755229,1.000000,0.000000,-0.011392,0.011392,10,10,1.000000,1.000000
5,0.006329,-0.020253,0.032911,57,52,0.701811,1.000000,-0.010127,-0.026582,0.006329,19,27,0.301996,0.905987
10,-0.011392,-0.040506,0.017722,63,72,0.491262,1.000000,-0.026582,-0.045570,-0.007595,19,40,0.008641,0.034566


## 5) Figures (Plotly inline)

In [ ]:

def ci_to_errors(means, ci):
    xs = list(means.keys())
    y  = np.array([means[b] for b in xs])
    lo = np.array([ci[b][0] for b in xs])
    hi = np.array([ci[b][1] for b in xs])
    return xs, y, hi - y, y - lo

# 1) TI vs β
xs, y, arr, arrminus = ci_to_errors(rates_TI, ci_TI)
fig_ti = go.Figure()
fig_ti.add_trace(go.Scatter(x=xs, y=y, mode='lines+markers',
                            error_y=dict(type='data', array=arr, arrayminus=arrminus, visible=True)))
fig_ti.update_layout(title="Dose–response: TI vs β (α=15, K=48, COM)", xaxis_title="β", yaxis_title="True×Informative")
fig_ti.show()

# 2) H vs β
xs, y, arr, arrminus = ci_to_errors(rates_H, ci_H)
fig_h = go.Figure()
fig_h.add_trace(go.Scatter(x=xs, y=y, mode='lines+markers',
                           error_y=dict(type='data', array=arr, arrayminus=arrminus, visible=True)))
fig_h.update_layout(title="Dose–response: H vs β", xaxis_title="β", yaxis_title="Hallucination rate H")
fig_h.show()

# 3) U vs β (Wilson CI)
xs = BETA_ORDER
y = [U[b] for b in xs]
lo = [U_ci[b][0] for b in xs]
hi = [U_ci[b][1] for b in xs]
arr = np.array(hi) - np.array(y)
arrminus = np.array(y) - np.array(lo)
fig_u = go.Figure()
fig_u.add_trace(go.Scatter(x=xs, y=y, mode='lines+markers',
                           error_y=dict(type='data', array=arr, arrayminus=arrminus, visible=True)))
fig_u.update_layout(title="Dose–response: U vs β", xaxis_title="β", yaxis_title="U (Unnecessary abstention)")
fig_u.show()

# 4) Four-bucket stacked bars
bucket_levels = ["T&I","T&¬I","¬T&I","¬T&¬I"]
counts_per_beta = {}
for b in BETA_ORDER:
    buckets = [df_map[b][q]["bucket"] for q in qids]
    counts = Counter(buckets)
    counts_per_beta[b] = [counts.get(l,0) for l in bucket_levels]
props = {b: np.array(counts_per_beta[b]) / sum(counts_per_beta[b]) for b in BETA_ORDER}
fig_stack = go.Figure()
for i, lab in enumerate(bucket_levels):
    fig_stack.add_trace(go.Bar(name=lab, x=BETA_ORDER, y=[props[b][i] for b in BETA_ORDER]))
fig_stack.update_layout(barmode='stack', title="Four-bucket breakdown (pooled held-out folds)", xaxis_title="β", yaxis_title="Proportion")
fig_stack.show()

# 5) Transition butterflies (β=1 → β∈{0,2,5,10})
def transitions(base_df, tgt_df, qids):
    trans = Counter()
    for q in qids:
        a = base_df[q]["bucket"]; b = tgt_df[q]["bucket"]
        trans[(a,b)] += 1
    return trans

for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
    trans = transitions(df_map[BASELINE_BETA], df_map[b], qids)
    flows = [("T&I","T&¬I"), ("¬T&I","T&I"), ("T&I","¬T&I"), ("¬T&I","T&¬I")]
    ycats = [f"{a}→{c}" for (a,c) in flows]
    vals = [trans.get((a,c), 0) for (a,c) in flows]
    x_vals = [-vals[0], vals[1], -vals[2], vals[3]]
    fig_bfly = go.Figure()
    fig_bfly.add_trace(go.Bar(y=ycats, x=x_vals, orientation='h'))
    fig_bfly.update_layout(title=f"Transitions β=1 → β={b}", xaxis_title="Count (left negative, right positive)")
    fig_bfly.add_vline(x=0, line_dash="dash")
    fig_bfly.show()


## 6) β=1 replication across seeds (descriptive only)

In [7]:

if "seed" in df.columns:
    b1 = df[df["beta"]==BASELINE_BETA].copy()
    if len(b1)>0 and b1["seed"].notna().any():
        if use_binary:
            rep = b1.groupby("seed")[[TRUTH_BIN_COL, INFO_BIN_COL]].mean().rename(
                columns={TRUTH_BIN_COL:"truth_bin_mean", INFO_BIN_COL:"info_bin_mean"})
        else:
            rep = b1.groupby("seed")[[TRUTH_PROB_COL, INFO_PROB_COL]].mean().rename(
                columns={TRUTH_PROB_COL:"truth_prob_mean", INFO_PROB_COL:"info_prob_mean"})
        print("β=1 replication (by seed):")
        display(rep)
else:
    print("No seed column to show replication.")


β=1 replication (by seed):


,truth_bin_mean,info_bin_mean
seed,,
42,0.920253,0.594937


## 7) Optional aggregates from `summary_dump` (MC1/MC2/CE/KL)

In [8]:
## Load summary_dump aggregates (MC1/MC2/CE Loss/KL)

def parse_meta_from_name(name):
    m_alpha = re.search(r'(?:^|[_-])alpha[_-](\d+)', name)
    m_fold  = re.search(r'(?:^|[_-])fold[_-](\d+)', name)
    m_beta  = re.search(r'(?:^|[_-])beta[_-]([0-9]+(?:\.[0-9]+)?)', name)
    alpha = int(m_alpha.group(1)) if m_alpha else None
    fold  = int(m_fold.group(1)) if m_fold else None
    beta  = float(m_beta.group(1)) if m_beta else 1.0
    return alpha, fold, beta


records = []
for g in SUMMARY_GLOBS:
    for path in glob.glob(g):
        base = os.path.basename(path)
        try:
            tdf = pd.read_csv(path)
        except Exception as e:
            print("Skipping unreadable summary_dump:", base, e); continue
        alpha, fold, beta = parse_meta_from_name(base)
        row = {"alpha": alpha, "fold": fold, "beta": beta, "file": base}
        cols = [c.strip() for c in tdf.columns]
        if "MC1" in cols and "MC2" in cols and "GPT-judge acc" in cols:
            row["MC1"] = float(tdf.loc[0, "MC1"])
            row["MC2"] = float(tdf.loc[0, "MC2"])
            row["GPT-judge acc"] = float(tdf.loc[0, "GPT-judge acc"])
            if "GPT-info acc" in tdf.columns:
                row["GPT-info acc"] = float(tdf.loc[0, "GPT-info acc"])
        if "CE Loss" in cols and "KL wrt Orig" in cols:
            row["CE Loss"] = float(tdf.loc[0, "CE Loss"])
            row["KL wrt Orig"] = float(tdf.loc[0, "KL wrt Orig"])
        records.append(row)

summary_dump = pd.DataFrame.from_records(records)
summary_dump.sort_values("file").drop_duplicates(subset=["alpha","fold","beta"], keep="last")



# Average summary_dump metrics across folds, indexed by beta
if not summary_dump.empty:
    # Only average numeric columns, group by beta
    numeric_cols = summary_dump.select_dtypes(include=[np.number]).columns.difference(['fold', 'alpha', 'beta'])
    avg_summary = summary_dump.groupby("beta")[numeric_cols].mean()
    display(avg_summary)
else:
    print("No summary_dump data to average.")

,CE Loss,GPT-info acc,GPT-judge acc,KL wrt Orig,MC1,MC2
beta,,,,,,
0.0,2.717855,0.583544,0.918987,0.0,0.372152,0.572919
1.0,2.717855,0.594937,0.920253,0.0,0.372152,0.572919
2.0,2.717855,0.598734,0.920253,0.0,0.372152,0.572919
5.0,2.717855,0.591139,0.930380,0.0,0.372152,0.572919
10.0,2.717855,0.556962,0.945570,0.0,0.372152,0.572919


# Soft-Label Analyses Add-on (Frequentist + Bayesian)

This block adds:
1) **Proper scoring rules** (paired Brier & Log-loss) for **Truth** and **Informativeness** — with paired bootstrap CIs and Wilcoxon signed-rank p-values (vs β=1).
2) **Bayesian bootstrap** over items to estimate:
   - Pr(ΔTI > 0) using **expected TI** from probabilities,
   - Pr(ΔH < 0) using **expected H** from probabilities.
3) **Calibration checks** by β (reliability curves + ECE) for Truth and Informativeness (Plotly figures).
4) *(Optional)* **Soft-U** sensitivity (expected U using probabilities) with Bayesian bootstrap CIs.

**Assumptions / dependencies from earlier notebook cells:**
- `df` contains pooled held-out rows across folds with columns:
  - Binary: `TRUTH_BIN_COL`, `INFO_BIN_COL` (0/1).
  - Probabilities: `TRUTH_PROB_COL`, `INFO_PROB_COL` in [0,1].
  - Metadata: `beta`, `Question`, and optional `seed`.
- `BETA_ORDER`, `BASELINE_BETA`, `PRIMARY_SEED` are set.
- Plotly is available and figures should be shown inline (no saving).


In [9]:
# ---- Safety checks on required columns ----
req_cols = [TRUTH_PROB_COL, INFO_PROB_COL, TRUTH_BIN_COL, INFO_BIN_COL, "beta", "Question"]
missing = [c for c in req_cols if c not in df.columns]
assert not missing, f"Missing required columns for soft-label analysis: {missing}"

# ---- Helper: choose deterministically one seed per β (same rule as in the core analysis) ----
def _choose_seed_for_beta(sub, beta):
    if "seed" in sub.columns and sub["seed"].notna().any():
        seeds = sorted(sub["seed"].dropna().unique().tolist())
        if beta == BASELINE_BETA and (PRIMARY_SEED is not None) and (PRIMARY_SEED in seeds):
            return PRIMARY_SEED
        # otherwise choose smallest seed deterministically
        return seeds[0]
    return None

# ---- Extract a tidy frame per β with paired data: (Question, yT, yI, pT, pI) ----
def get_beta_frame(beta: float) -> pd.DataFrame:
    sub = df[df["beta"] == float(beta)].copy()
    chosen = _choose_seed_for_beta(sub, beta)
    if chosen is not None:
        sub = sub[sub["seed"] == chosen].copy()
    # Ensure numeric types
    sub = sub.copy()
    sub[TRUTH_BIN_COL] = sub[TRUTH_BIN_COL].astype(int)
    sub[INFO_BIN_COL]  = sub[INFO_BIN_COL].astype(int)
    sub[TRUTH_PROB_COL] = sub[TRUTH_PROB_COL].astype(float).clip(0.0, 1.0)
    sub[INFO_PROB_COL]  = sub[INFO_PROB_COL].astype(float).clip(0.0, 1.0)

    out = sub[["Question", TRUTH_BIN_COL, INFO_BIN_COL, TRUTH_PROB_COL, INFO_PROB_COL]].rename(
        columns={
            TRUTH_BIN_COL: "yT",
            INFO_BIN_COL: "yI",
            TRUTH_PROB_COL: "pT",
            INFO_PROB_COL: "pI",
        }
    ).dropna(subset=["Question"])
    return out

# ---- Pair questions across all β as in the core analysis ----
frames_by_beta_soft = {b: get_beta_frame(b) for b in BETA_ORDER}
common_qs = None
for b, d in frames_by_beta_soft.items():
    s = set(d["Question"])
    common_qs = s if common_qs is None else (common_qs & s)
paired_qids_soft = sorted(list(common_qs))
assert len(paired_qids_soft) > 0, "No overlapping questions across β for soft-label analyses."

# Index frames to paired set
for b in BETA_ORDER:
    frames_by_beta_soft[b] = frames_by_beta_soft[b][frames_by_beta_soft[b]["Question"].isin(paired_qids_soft)].set_index("Question").sort_index()


## 1) Proper Scoring Rules (Paired) — Brier & Log-loss for T and I
We compare each β to baseline (β=1) using per-item **score differences**:
- **Brier**: (p − y)^2
- **Log-loss**: −[y·log(p) + (1−y)·log(1−p)], with clipping for stability

We compute:
- Mean Δ (β − baseline)
- 95% paired bootstrap CI on the mean Δ
- Wilcoxon signed-rank p-value on per-item Δ

In [11]:
from numpy.random import default_rng
from scipy.stats import wilcoxon

def brier(y, p):
    return (p - y)**2

def log_loss(y, p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return -(y*np.log(p) + (1-y)*np.log(1-p))

def paired_bootstrap_ci(deltas, n_boot=10000, seed=2025):
    rng = default_rng(seed)
    n = len(deltas)
    means = []
    idx = np.arange(n)
    for _ in range(n_boot):
        b = rng.choice(idx, size=n, replace=True)
        means.append(np.mean(deltas[b]))
    lo, hi = np.percentile(means, [2.5, 97.5])
    return float(np.mean(deltas)), float(lo), float(hi)

def compute_scoring_comparison(metric_name, y_col, p_col):
    base = frames_by_beta_soft[BASELINE_BETA]
    y_base = base[y_col].values
    p_base = base[p_col].values
    res_rows = []
    for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
        tgt = frames_by_beta_soft[b].reindex(base.index)  # align by Question
        y_tgt = tgt[y_col].values
        p_tgt = tgt[p_col].values

        # Sanity: y should match baseline ground-truth (same questions & labels)
        # (Judges are the same; we keep y from tgt just for completeness)
        # Compute per-item deltas vs baseline
        if metric_name == "brier":
            s_base = brier(y_base, p_base)
            s_tgt  = brier(y_tgt,  p_tgt)
        elif metric_name == "logloss":
            s_base = log_loss(y_base, p_base)
            s_tgt  = log_loss(y_tgt,  p_tgt)
        else:
            raise ValueError("Unknown metric_name")

        deltas = s_tgt - s_base
        mean_delta, lo, hi = paired_bootstrap_ci(deltas)
        try:
            w_p = wilcoxon(deltas, zero_method="wilcox", alternative="two-sided").pvalue
        except Exception:
            w_p = np.nan

        res_rows.append(dict(
            beta=b, metric=metric_name, target=y_col,
            mean_delta=mean_delta, ci_lo=lo, ci_hi=hi, wilcoxon_p=w_p
        ))
    return pd.DataFrame(res_rows).set_index(["metric","target","beta"]).sort_index()

# Truth scoring comparisons
score_truth_brier  = compute_scoring_comparison("brier",  y_col="yT", p_col="pT")
score_truth_ll     = compute_scoring_comparison("logloss",y_col="yT", p_col="pT")
# Informativeness scoring comparisons
score_info_brier   = compute_scoring_comparison("brier",  y_col="yI", p_col="pI")
score_info_ll      = compute_scoring_comparison("logloss",y_col="yI", p_col="pI")

scoring_table = pd.concat([score_truth_brier, score_truth_ll, score_info_brier, score_info_ll])
display(scoring_table.round(5))


mean_delta    ci_lo    ci_hi  wilcoxon_p
metric  target beta                                          
brier   yT     0       -0.00010 -0.00128  0.00106     0.84552
               2       -0.00096 -0.00215  0.00011     0.01683
               5        0.00003 -0.00165  0.00169     0.10484
               10      -0.00028 -0.00194  0.00133     0.00880
logloss yT     0       -0.00047 -0.00420  0.00315     0.79997
               2       -0.00364 -0.00739 -0.00016     0.01862
               5        0.00015 -0.00524  0.00544     0.11966
               10      -0.00164 -0.00697  0.00347     0.00796
brier   yI     0       -0.00020 -0.00131  0.00091     0.38726
               2       -0.00064 -0.00181  0.00033     0.01929
               5        0.00017 -0.00133  0.00159     0.00291
               10       0.00079 -0.00083  0.00253     0.00001
logloss yI     0       -0.00078 -0.00410  0.00262     0.35716
               2       -0.00115 -0.00466  0.00192     0.01760
               5        0.00154 -0.00324  0.00609     0.00253
               10       0.00364 -0.00161  0.00927     0.00001

## 2) Bayesian Bootstrap on Expected TI / H
We compute per-item **expected** outcomes from probabilities:
- **E[TI] = pT · pI**
- **E[H]  = (1 − pT) · pI**
For each β, we form per-item deltas vs baseline and run a **Bayesian bootstrap** to get:
- Posterior for the **mean ΔTI** and **Pr(ΔTI > 0)**
- Posterior for the **mean ΔH**  and **Pr(ΔH < 0)**

In [12]:
# %%
def bayes_bootstrap_weights(n, rng):
    # Dirichlet(1,...,1) via Gamma(1,1) normalized
    w = rng.gamma(shape=1.0, scale=1.0, size=n)
    return w / w.sum()

def bayes_bootstrap_posterior(delta, B=5000, seed=2025):
    rng = default_rng(seed)
    n = len(delta)
    post_means = np.empty(B, dtype=float)
    for b in range(B):
        w = bayes_bootstrap_weights(n, rng)
        post_means[b] = float(np.sum(w * delta))
    return post_means

def expected_metrics_from_probs(frame):
    # frame: index=Question, columns yT,yI,pT,pI
    pT = frame["pT"].values
    pI = frame["pI"].values
    e_TI = pT * pI
    e_H  = (1.0 - pT) * pI
    return e_TI, e_H

bayes_rows = []
base_frame = frames_by_beta_soft[BASELINE_BETA]

eTI_base, eH_base = expected_metrics_from_probs(base_frame)
for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
    tgt = frames_by_beta_soft[b].reindex(base_frame.index)
    eTI_tgt, eH_tgt = expected_metrics_from_probs(tgt)

    dTI = eTI_tgt - eTI_base
    dH  = eH_tgt  - eH_base

    # Bayesian bootstrap posteriors
    post_dTI = bayes_bootstrap_posterior(dTI, B=5000, seed=2025)
    post_dH  = bayes_bootstrap_posterior(dH,  B=5000, seed=2026)

    pr_dTI_pos = float((post_dTI > 0).mean())
    pr_dH_neg  = float((post_dH  < 0).mean())

    dTI_ci = (float(np.quantile(post_dTI, 0.025)), float(np.quantile(post_dTI, 0.975)))
    dH_ci  = (float(np.quantile(post_dH,  0.025)), float(np.quantile(post_dH,  0.975)))

    bayes_rows.append(dict(
        beta=b,
        dTI_mean=float(post_dTI.mean()), dTI_lo=dTI_ci[0], dTI_hi=dTI_ci[1], Pr_dTI_gt0=pr_dTI_pos,
        dH_mean=float(post_dH.mean()),   dH_lo=dH_ci[0],   dH_hi=dH_ci[1],   Pr_dH_lt0=pr_dH_neg
    ))

bayes_bb_table = pd.DataFrame(bayes_rows).set_index("beta").sort_index()
display(bayes_bb_table.round(5))


,dTI_mean,dTI_lo,dTI_hi,Pr_dTI_gt0,dH_mean,dH_lo,dH_hi,Pr_dH_lt0
beta,,,,,,,,
0,-0.00975,-0.02387,0.00361,0.0786,-0.00144,-0.01266,0.00937,0.6084
2,0.00493,-0.00943,0.01953,0.7520,-0.00045,-0.01071,0.00978,0.5302
5,0.00664,-0.01696,0.02985,0.7092,-0.00993,-0.02601,0.00578,0.8970
10,-0.01165,-0.03860,0.01511,0.1968,-0.02765,-0.04484,-0.01023,0.9978


## 3) Calibration Checks by β (Reliability + ECE)
For **Truth** and **Informativeness**:
- Make reliability curves by binning **probabilities** into deciles and plotting **mean predicted vs empirical rate** (using binary labels).
- Compute **ECE** (Expected Calibration Error) as ∑_bins |acc−conf|·(n/N).

*Note:* We do **not** recalibrate here; this is diagnostic to see if calibration differs across β.


In [13]:
def reliability_by_beta(frames_by_beta, y_col, p_col, nbins=10):
    """Return dict[beta] -> (df_bins, ECE, N) for a given outcome."""
    out = {}
    for b, fr in frames_by_beta.items():
        p = fr[p_col].values
        y = fr[y_col].values
        N = len(y)
        bins = np.linspace(0, 1, nbins+1)
        idx = np.digitize(p, bins, right=True)
        # clamp
        idx[idx < 1] = 1
        idx[idx > nbins] = nbins
        rows = []
        ece = 0.0
        for k in range(1, nbins+1):
            mask = (idx == k)
            n = int(mask.sum())
            if n == 0:
                rows.append(dict(bin=k, n=0, conf=np.nan, acc=np.nan))
                continue
            conf = float(np.mean(p[mask]))
            acc  = float(np.mean(y[mask]))
            rows.append(dict(bin=k, n=n, conf=conf, acc=acc))
            ece += abs(acc - conf) * (n / N)
        out[b] = (pd.DataFrame(rows), float(ece), int(N))
    return out

# Compute reliability for Truth and Info
rel_T = reliability_by_beta(frames_by_beta_soft, "yT", "pT", nbins=10)
rel_I = reliability_by_beta(frames_by_beta_soft, "yI", "pI", nbins=10)

def plot_reliability(rel_dict, title):
    fig = go.Figure()
    # 45-degree reference
    fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Ideal", line=dict(dash="dash")))
    for b in BETA_ORDER:
        bins_df, ece, N = rel_dict[b]
        bins_df = bins_df.dropna()
        if len(bins_df)==0: 
            continue
        fig.add_trace(go.Scatter(
            x=bins_df["conf"], y=bins_df["acc"],
            mode="lines+markers", name=f"β={b} (ECE={ece:.3f}, N={N})"
        ))
    fig.update_layout(title=title, xaxis_title="Mean predicted prob", yaxis_title="Empirical rate")
    fig.show()

plot_reliability(rel_T, "Reliability (Truth) by β")
plot_reliability(rel_I, "Reliability (Informativeness) by β")

# Tabular ECE summary
ece_rows = []
for b in BETA_ORDER:
    ece_rows.append(dict(beta=b, ECE_T=rel_T[b][1], N_T=rel_T[b][2], ECE_I=rel_I[b][1], N_I=rel_I[b][2]))
ece_table = pd.DataFrame(ece_rows).set_index("beta").sort_index()
display(ece_table.round(4))


,ECE_T,N_T,ECE_I,N_I
beta,,,,
0,0.0082,790,0.0084,790
1,0.0086,790,0.0091,790
2,0.0057,790,0.0084,790
5,0.0088,790,0.0106,790
10,0.0072,790,0.0122,790


## 4) Optional: Soft-U Sensitivity (Expected U via probabilities)
**Definition:** Among baseline (β=1) items that are *binary* TI, estimate the **expected** fraction that become **T&¬I** at β using probabilities:

`soft_U_β = mean_{q in TI_base} [ E[T_β · (1 − I_β)] ] = mean[ pT_β · (1 − pI_β) ]`

We also compute a Bayesian bootstrap 95% CrI and, if you set a tolerance `Y_tol`, the posterior **Pr(soft_U ≤ Y_tol)**.


In [14]:
# %%
def soft_U_expected(beta, Y_tol=None, B=5000, seed=2027):
    base = frames_by_beta_soft[BASELINE_BETA]
    # Identify TI items at baseline using **binary** labels
    ti_mask = (base["yT"].values == 1) & (base["yI"].values == 1)
    q_TI = base.index[ti_mask]
    n = len(q_TI)
    if n == 0:
        return dict(beta=beta, soft_U=np.nan, ci_lo=np.nan, ci_hi=np.nan, Pr_le_tol=np.nan, n=0)

    tgt = frames_by_beta_soft[beta].reindex(q_TI)
    pT = tgt["pT"].values
    pI = tgt["pI"].values
    per_item = pT * (1.0 - pI)  # E[T & ¬I] at β

    # Bayesian bootstrap
    rng = default_rng(seed)
    post = []
    for _ in range(B):
        w = bayes_bootstrap_weights(n, rng)
        post.append(float(np.sum(w * per_item)))
    post = np.array(post, float)
    mean = float(post.mean())
    lo, hi = float(np.quantile(post, 0.025)), float(np.quantile(post, 0.975))
    pr_le_tol = float((post <= Y_tol).mean()) if (Y_tol is not None) else np.nan
    return dict(beta=beta, soft_U=mean, ci_lo=lo, ci_hi=hi, Pr_le_tol=pr_le_tol, n=n)

# Example usage (set a tolerance if you have one, e.g., 0.10 for 10%)
Y_tol = None  # e.g., 0.10
softu_rows = []
for b in [bb for bb in BETA_ORDER if bb != BASELINE_BETA]:
    softu_rows.append(soft_U_expected(b, Y_tol=Y_tol, B=5000, seed=2027+b))
softU_table = pd.DataFrame(softu_rows).set_index("beta").sort_index()
display(softU_table.round(4))


,soft_U,ci_lo,ci_hi,Pr_le_tol,n
beta,,,,,
0,0.0324,0.0188,0.0505,NaN,407
2,0.0269,0.0146,0.0430,NaN,407
5,0.0864,0.0614,0.1151,NaN,407
10,0.1387,0.1080,0.1735,NaN,407


### Interpretation Tips
- **Scoring rules (ΔBrier, ΔLog-loss)**: negative mean Δ = improvement (lower is better for both). Check paired CIs and Wilcoxon p’s.
- **Bayesian bootstrap (ΔTI, ΔH)**:
  - **Pr(ΔTI > 0)** close to 0.5 ⇒ inconclusive; above ~0.9 ⇒ decent evidence TI increases.
  - **Pr(ΔH < 0)** close to 1 ⇒ strong evidence of fewer hallucinations.
- **ECE & reliability**: lower ECE is better; curves closer to diagonal are better calibrated. If calibration **differs by β**, consider reporting both raw and *recalibrated* results (fit calibration on a separate split).
- **Soft-U**: complements the hard **U** by showing expected behavior using probabilities. Use together; don’t replace the deployable binary U.


## Plots

In [30]:
# ### Figure 1 (Option A): One multi-panel dose–response figure (TI, H, U) with shared β axis
# This cell composes the three dose–response plots into a single vertical, 3-panel figure
# and saves it as both HTML (interactive) and PNG (static).
# - Inputs expected from earlier cells:
#   - BETA_ORDER (ordered list of β)
#   - rates_TI, ci_TI  (dicts: beta -> mean / (lo, hi))
#   - rates_H,  ci_H   (dicts: beta -> mean / (lo, hi))
#   - U, U_ci          (dicts: beta -> mean / (lo, hi))
# - Outputs:
#   - fig_dose_response_ti_h_u.html  (interactive, good for Distill)
#   - fig_dose_response_ti_h_u.png   (static preview; requires kaleido)

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def _ci_to_errors(means: dict, ci: dict, xs):
    y  = np.array([means[b] for b in xs], dtype=float)
    lo = np.array([ci[b][0] for b in xs], dtype=float)
    hi = np.array([ci[b][1] for b in xs], dtype=float)
    err_plus  = hi - y
    err_minus = y - lo
    return y, err_plus, err_minus

# Prepare data in β order
xs = list(BETA_ORDER)

# TI
y_ti, e_ti_plus, e_ti_minus = _ci_to_errors(rates_TI, ci_TI, xs)

# H
y_h,  e_h_plus,  e_h_minus  = _ci_to_errors(rates_H,  ci_H,  xs)

# U (Wilson CI)
y_u  = np.array([U[b] for b in xs], dtype=float)
lo_u = np.array([U_ci[b][0] for b in xs], dtype=float)
hi_u = np.array([U_ci[b][1] for b in xs], dtype=float)
e_u_plus  = hi_u - y_u
e_u_minus = y_u  - lo_u

# Build multi-panel figure
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.06,
    subplot_titles=("Dose–response: TI vs β (α=15, K=48, COM)",
                    "Dose–response: H vs β",
                    "Dose–response: U vs β")
)

# Consistent styling
marker_kwargs = dict(size=8, line=dict(width=1))
err_kwargs    = lambda arr, arrminus: dict(type="data", array=arr, arrayminus=arrminus, visible=True)

# Panel 1: TI
fig.add_trace(go.Scatter(
    x=xs, y=y_ti, mode="lines+markers", name="TI",
    marker=dict(color="#7a3db8", **marker_kwargs),
    line=dict(color="#7a3db8"),
    error_y=err_kwargs(e_ti_plus, e_ti_minus)
), row=1, col=1)

# Panel 2: H
fig.add_trace(go.Scatter(
    x=xs, y=y_h, mode="lines+markers", name="H",
    marker=dict(color="#2a6fdb", **marker_kwargs),
    line=dict(color="#2a6fdb"),
    error_y=err_kwargs(e_h_plus, e_h_minus)
), row=2, col=1)

# Panel 3: U
fig.add_trace(go.Scatter(
    x=xs, y=y_u, mode="lines+markers", name="U",
    marker=dict(color="#1aa69a", **marker_kwargs),
    line=dict(color="#1aa69a"),
    error_y=err_kwargs(e_u_plus, e_u_minus)
), row=3, col=1)

# Axes labels & layout
fig.update_xaxes(title_text="β", row=3, col=1)
fig.update_yaxes(title_text="True×Informative (TI)", row=1, col=1)
fig.update_yaxes(title_text="Hallucination rate H",   row=2, col=1)
fig.update_yaxes(title_text="U (Unnecessary abstention)", row=3, col=1)

fig.update_layout(
    height=900, width=1100,
    showlegend=False,
    margin=dict(l=70, r=30, t=80, b=60),
)

fig.show()

# Save to HTML and PNG
OUTPUT_BASENAME = "fig_dose_response_ti_h_u"
fig.write_html(f"{OUTPUT_BASENAME}.html", include_plotlyjs="cdn", full_html=True)

# PNG export requires the 'kaleido' package: pip install -U kaleido
try:
    fig.write_image(f"{OUTPUT_BASENAME}.png", scale=2, width=1100, height=900)
    print(f"Saved {OUTPUT_BASENAME}.html and {OUTPUT_BASENAME}.png")
except Exception as e:
    print(f"HTML saved. PNG export skipped (install 'kaleido' to enable). Details: {e}")


HTML saved. PNG export skipped (install 'kaleido' to enable). Details: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido



In [31]:
# %% [markdown]
# ### Four-bucket stacked bars (β as ordered category) — with optional % labels

from collections import Counter
import numpy as np
import plotly.graph_objects as go

# Config
bucket_levels = ["T&I", "T&¬I", "¬T&I", "¬T&¬I"]
bucket_colors = {
    "T&I":   "#5b6df8",  # blue
    "T&¬I":  "#f04d3a",  # red/orange
    "¬T&I":  "#12c8a0",  # teal/green
    "¬T&¬I": "#8c6df0",  # purple
}
show_percent_labels = True  # set False to hide text inside bars

# Count buckets per β
counts_per_beta = {}
for b in BETA_ORDER:
    buckets = [df_map[b][q]["bucket"] for q in qids]
    counts = Counter(buckets)
    counts_per_beta[b] = [counts.get(level, 0) for level in bucket_levels]

# Convert to proportions per β
props = {
    b: (np.array(counts_per_beta[b], dtype=float) /
        max(1.0, float(sum(counts_per_beta[b]))))
    for b in BETA_ORDER
}

# β as ordered categorical along x
x_categories = [str(b) for b in BETA_ORDER]

fig_stack = go.Figure()

# Add one trace per bucket level
for i, lab in enumerate(bucket_levels):
    y_vals = [props[b][i] for b in BETA_ORDER]
    text_vals = [f"{v:.0%}" for v in y_vals] if show_percent_labels else None

    fig_stack.add_trace(go.Bar(
        name=lab,
        x=x_categories,
        y=y_vals,
        text=text_vals,
        textposition="inside" if show_percent_labels else "none",
        marker_color=bucket_colors.get(lab, None),
        hovertemplate=f"β=%{{x}}<br>{lab}: %{{y:.3%}}<extra></extra>"
    ))

fig_stack.update_layout(
    barmode='stack',
    title="Four-bucket breakdown (pooled held-out folds)",
    xaxis=dict(
        title="β",
        type="category",
        categoryorder="array",
        categoryarray=x_categories
    ),
    yaxis=dict(
        title="Proportion",
        rangemode="tozero",
        tickformat=".0%"  # display 0–100% ticks
    ),
    legend=dict(title=None),
    bargap=0.15,
    margin=dict(l=70, r=30, t=60, b=60),
    height=420,
)

fig_stack.show()
# Save to HTML and PNG
OUT_BASENAME = "four-bucket-breakdown"
fig_stack.write_html(f"{OUT_BASENAME}.html", include_plotlyjs="cdn", full_html=True)
try:
    fig_stack.write_image(f"{OUT_BASENAME}.png", scale=2, width=1100, height=300)
    print(f"Saved {OUT_BASENAME}.html and {OUT_BASENAME}.png")
except Exception as e:
    print(f"HTML saved. PNG export skipped (install 'kaleido'). Details: {e}")


HTML saved. PNG export skipped (install 'kaleido'). Details: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido



In [32]:
# %% [markdown]
# ### Figure: Transition “butterflies” stacked (β=1 → β∈{…}) — FIXED hovertemplate formatting
# Stacks all butterflies into one vertical multi-panel figure with a shared, symmetric x-axis.

from collections import Counter
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def transitions(base_df, tgt_df, qids):
    trans = Counter()
    for q in qids:
        a = base_df[q]["bucket"]; b = tgt_df[q]["bucket"]
        trans[(a, b)] += 1
    return trans

flows = [
    ("T&I","T&¬I"),   # cost: TI -> T&¬I
    ("¬T&I","T&I"),   # benefit: ¬T&I -> T&I
    ("T&I","¬T&I"),   # cost: TI -> ¬T&I
    ("¬T&I","T&¬I")   # benefit (safety): ¬T&I -> T&¬I
]
ycats = [f"{a}→{c}" for (a,c) in flows]
cost_idx    = {0, 2}
benefit_idx = {1, 3}

target_betas = [bb for bb in BETA_ORDER if bb != BASELINE_BETA]

# Precompute range for symmetric x-axis
all_abs = []
per_beta_vals = {}
for b in target_betas:
    trans = transitions(df_map[BASELINE_BETA], df_map[b], qids)
    vals = [trans.get((a,c), 0) for (a,c) in flows]
    per_beta_vals[b] = vals
    all_abs.extend([abs(v) for v in vals])

xmax = max(all_abs) if all_abs else 1
xrange = [-xmax * 1.05, xmax * 1.05]

fig_bfly = make_subplots(
    rows=len(target_betas), cols=1, shared_xaxes=True, vertical_spacing=0.08,
    subplot_titles=[f"Transitions β=1 → β={b}" for b in target_betas]
)

col_cost    = "#f04d3a"
col_benefit = "#12c8a0"

for r, b in enumerate(target_betas, start=1):
    vals = per_beta_vals[b]
    x_neg = [-(vals[i]) if i in cost_idx else 0 for i in range(len(flows))]
    x_pos = [ (vals[i]) if i in benefit_idx else 0 for i in range(len(flows))]

    # Use f-strings and escape Plotly placeholders by doubling braces
    hover_cost    = f"β={b}<br>%{{y}}: %{{x:.0f}}<extra></extra>"
    hover_benefit = f"β={b}<br>%{{y}}: %{{x:.0f}}<extra></extra>"

    fig_bfly.add_trace(go.Bar(
        y=ycats, x=x_neg, orientation="h", name="Cost (away from TI)",
        marker_color=col_cost, hovertemplate=hover_cost
    ), row=r, col=1)

    fig_bfly.add_trace(go.Bar(
        y=ycats, x=x_pos, orientation="h", name="Benefit/Safety",
        marker_color=col_benefit, hovertemplate=hover_benefit
    ), row=r, col=1)

    fig_bfly.add_vline(x=0, line_dash="dash", line_color="#666", row=r, col=1)

fig_bfly.update_layout(
    barmode="overlay",
    height=260 * len(target_betas) + 120,
    width=1000,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1.0, title=None),
    margin=dict(l=120, r=30, t=70, b=60),
)

for r in range(1, len(target_betas)+1):
    fig_bfly.update_xaxes(range=xrange, title_text="Count (left = cost, right = benefit)", row=r, col=1)
    fig_bfly.update_yaxes(title_text=None, row=r, col=1, categoryorder="array", categoryarray=ycats)

fig_bfly.show()

# Save to HTML and PNG
OUT_BASENAME = "fig_transitions_butterflies_stacked"
fig_bfly.write_html(f"{OUT_BASENAME}.html", include_plotlyjs="cdn", full_html=True)
try:
    fig_bfly.write_image(f"{OUT_BASENAME}.png", scale=2, width=1000, height=260 * len(target_betas) + 120)
    print(f"Saved {OUT_BASENAME}.html and {OUT_BASENAME}.png")
except Exception as e:
    print(f"HTML saved. PNG export skipped (install 'kaleido'). Details: {e}")


HTML saved. PNG export skipped (install 'kaleido'). Details: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

